In [2]:
import pandas as pd
import numpy as np
import re
import string
import time
from datetime import datetime
from datetime import date
import os

#Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import metrics

# Save model
import pickle
import joblib

# Shap Values
import shap


In [3]:
# fileLocation = 'C:\\Users\\VictorY\\Desktop\\TestData\\stage3FinalProcessedDF_Nov-17-2019.csv'
# moviesDataFileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Final_Data_Movies_Directors.csv'
directorDFfileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\cumulativeDirectorDF_Mar-21-2020.csv'
saveFileToPath = "C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\"

director_data = pd.read_csv(directorDFfileLocation)

In [4]:
director_data.head()

,director_ids,director_name,movies_produced,Average_rating_value,Average_rating_count,career_Length,Gender,birthYear,primaryProfession,knownForTitles,CareerStartYear,median_inter_event_time,average_movie_per_year,average_inter_event_time
0,nm0000005,Ingmar Bergman,70,7.36,6984.76,63,Male,1918,"writer,director,actor","tt0083922,tt0069467,tt0050986,tt0050976",1946,1.0,1.11,0.9
1,nm0000008,Marlon Brando,1,7.20,7002.00,1,Male,1924,"actor,soundtrack,director","tt0047296,tt0078788,tt0070849,tt0068646",1961,0.0,1.00,0.0
2,nm0000009,Richard Burton,1,5.70,676.00,1,Male,1925,"actor,producer,soundtrack","tt0061184,tt0057877,tt0059749,tt0087803",1967,0.0,1.00,0.0
3,nm0000010,James Cagney,1,5.80,162.00,1,Male,1899,"actor,soundtrack,director","tt0055256,tt0035575,tt0029870,tt0031867",1957,0.0,1.00,0.0
4,nm0000018,Kirk Douglas,2,5.85,722.50,3,Male,1916,"actor,producer,soundtrack","tt0054331,tt0052365,tt0080736,tt0049456",1973,2.0,0.67,2.0


In [5]:
required_features_director_data = ['movies_produced',
                                   'Average_rating_value',
                                   'Average_rating_count',
                                   'career_Length',
                                   'Gender',
                                   'birthYear',
                                   'CareerStartYear',
                                   'average_movie_per_year',
                                   'median_inter_event_time',
                                   'average_inter_event_time'
                                  ]

In [6]:
director_data = director_data[required_features_director_data]
print('The shape of our Director specific dataset is:', director_data.shape)

The shape of our Director specific dataset is: (41629, 10)


In [7]:
# Generic function to equally max split the dataframe with respect to the column Name, in our case the Gender column.

def max_equal_split_on_columnName(df,columnName='Gender'):
    df1 = pd.DataFrame()
    for i in df[columnName].value_counts().index:
        df1 = df1.append(df[(df[columnName]==i)].sample(n=df[columnName].value_counts().values.min(),replace = False))        
    return df1.reset_index(drop=True)

In [8]:
# Generic function to equally split the dataframe with respect to the column Name, in our case the Gender column.

def equal_split_on_columnName(nSampleEach, df,columnName='Gender'):
    df1 = pd.DataFrame()
    for i in df[columnName].value_counts().index:
        df1 = df1.append(df[(df[columnName]==i)].sample(n=nSampleEach,replace = False))        
    return df1.reset_index(drop=True)

In [9]:
def print_feature_importances(coef, names, top=-1):
    whole = []
    imp = coef
    imp,names = zip(*sorted(list(zip(imp, names)),reverse=True))
    for i in range(len(names)):
        string = "{} :: {}".format(str(names[i]),str(round(imp[i],3)))
        whole.append(string)
    return whole

In [10]:
def print_features_selected(df):
    print('[',end = '')
    for i,v in enumerate(df.columns.values):
        if(i==len(df.columns.values)-1):
            print("'" + str(v) +"']")
        else:
            print("'" + str(v) +"',")

In [11]:
model_features = ['movies_produced','Average_rating_value','Average_rating_count',
                  'career_Length','birthYear','CareerStartYear','average_movie_per_year',
                  'median_inter_event_time','average_inter_event_time']

In [12]:
temp_director_only = director_data.dropna(subset=model_features)
temp_director_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41629 entries, 0 to 41628
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   movies_produced           41629 non-null  int64  
 1   Average_rating_value      41629 non-null  float64
 2   Average_rating_count      41629 non-null  float64
 3   career_Length             41629 non-null  int64  
 4   Gender                    41629 non-null  object 
 5   birthYear                 41629 non-null  int64  
 6   CareerStartYear           41629 non-null  int64  
 7   average_movie_per_year    41629 non-null  float64
 8   median_inter_event_time   41629 non-null  float64
 9   average_inter_event_time  41629 non-null  float64
dtypes: float64(5), int64(4), object(1)
memory usage: 3.5+ MB


In [13]:
equalDF = equal_split_on_columnName(1000,temp_director_only)

In [14]:
# print_features_selected(temp_director_only)
print_features_selected(equalDF[model_features])

['movies_produced',
'Average_rating_value',
'Average_rating_count',
'career_Length',
'birthYear',
'CareerStartYear',
'average_movie_per_year',
'median_inter_event_time',
'average_inter_event_time']


In [15]:
start = time.time()
print("Started Reading files at :" + str(datetime.utcfromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S')))

Started Reading files at :2020-04-10 08:33:09


In [16]:
predictingFeature  =  ['Gender']
featuresSelected = equalDF.columns.to_list()
featuresSelected.remove('Gender')

In [17]:
# divide the data into attributes and labels
X = equalDF[featuresSelected]
y = equalDF[predictingFeature]

In [18]:
# Divide the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [19]:
# Training the model
# svclassifier = SVC(kernel='linear')
svclassifier = SVC(kernel='poly', degree=4,probability=True)
# svclassifier = SVC(kernel='rbf')
# svclassifier = SVC(kernel='sigmoid')

In [20]:
# Fit the model
svclassifier.fit(X_train, y_train)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=4, gamma='scale', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [21]:
train_score_SVC = svclassifier.score(X_train, y_train)
train_score_SVC

0.5075

In [22]:
test_score_SVC = svclassifier.score(X_test, y_test)
test_score_SVC

0.49

In [47]:
# Prediction based on the model trained
y_pred = svclassifier.predict(X_test)
# tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()

In [51]:
confusion_matrix(y_test,y_pred)

array([[194,   3],
       [201,   2]], dtype=int64)

In [52]:
# Classification Report
print("\nClassification Report\n")
print(classification_report(y_test, y_pred))


Classification Report

              precision    recall  f1-score   support

      Female       0.49      0.98      0.66       197
        Male       0.40      0.01      0.02       203

    accuracy                           0.49       400
   macro avg       0.45      0.50      0.34       400
weighted avg       0.44      0.49      0.33       400



In [53]:
row_to_show = 5
data_for_prediction = X_test.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

# svclassifier.predict(data_for_prediction_array)
svclassifier.predict_proba(data_for_prediction_array)

array([[0.5, 0.5]])

In [69]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
# explainer = shap.KernelExplainer(svclassifier)


svm_shap_values = shap.KernelExplainer(svclassifier.predict,X_test)


Using 400 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


TypeError: can't multiply sequence by non-int of type 'float'

## K-nearest Neighbours

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [56]:
def print_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == y_test)/len(y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

In [59]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [66]:
predictions = knn.predict
# predictions = predictions.reshape(400,1)


### Logicstic Classification

In [75]:
X.head(1)

,movies_produced,Average_rating_value,Average_rating_count,career_Length,birthYear,CareerStartYear,average_movie_per_year,median_inter_event_time,average_inter_event_time
0,14,6.96,22.0,28,1940,1984,0.5,0.0,2.08


In [76]:
y.head(1)

,Gender
0,Male


In [71]:
from sklearn.linear_model import LogisticRegression

In [89]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train.values.ravel())

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [90]:
y_pred = clf.predict(X_test)

In [97]:
train_score = clf.score(X_train, y_train)
train_score

0.548125

In [98]:
test_score = clf.score(X_test, y_test)
test_score

0.4975

### Naive Bayes

In [101]:
from sklearn.naive_bayes import GaussianNB

In [107]:
gnb = GaussianNB().fit(X_train, y_train.values.ravel())

In [108]:
y_pred = gnb.predict(X_test)

In [109]:
train_score_gnb = gnb.score(X_train, y_train)
train_score_gnb

0.555

In [110]:
test_score_gnb = gnb.score(X_test, y_test)
test_score_gnb

0.5375